In [925]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import math as m
import soundfile as sf

path_to_folder = os.getcwd()
path_to_voice = '\\LibriSpeech\\train-clean-100'
path_to_plane = '\\recordings\\'

In [926]:
def random_voice_file():
    path = path_to_folder+'\\LibriSpeech\\train-clean-100'
    dirr = os.listdir(path)
    dir1 = random.sample(dirr,1)[0]
    path = path + "\\" + dir1
    dir2 = os.listdir(path)
    dir2 = random.sample(dir2,1)[0]
    path = path + "\\" + dir2
    file = os.listdir(path)
    file = random.sample(file,1)[0]
    while file[-1] != 'c':
        file = os.listdir(path)
        file = random.sample(dir2,1)[0]
    path = path + "\\" + file
    return(path,file)

In [927]:
def random_plane_file():
    n = random.randint(0,24)
    return(path_to_folder+path_to_plane+"record_"+str(n)+".wav")

In [928]:
path,file = random_file()
print(path,file[:-5])
plane = random_plane_file()
print(plane)

d:\Utilisateurs\ENAC\projet_AIRBUS\LibriSpeech\train-clean-100\1116\132847\1116-132847-0030.flac 1116-132847-0030
d:\Utilisateurs\ENAC\projet_AIRBUS\recordings\record_24.wav


In [929]:
def get_time_steps(lendata,samplerate):
    return(np.linspace(0,int(lendata/samplerate),lendata))

In [930]:
import soundfile as sf
datavoice, sampleratevoice = sf.read(path)
dataplane, samplerateplane = sf.read(random_plane_file())
print(len(dataplane),len(datavoice))

5898240 97040


In [931]:
def f_exp(x,sigma):
    return m.exp(-1*((x)**2)/(2*(sigma**2)))

In [932]:
def filtre_voix_exp(data,place,sigma,samplerate):
    imax = int(len(data)*place)
    exp = [m.exp(-1*((x-imax)**2)/(2*(sigma**2))) for x in range(len(data))]
    tps = len([0 for k in range(len(data)) if exp[k]>0.001])/samplerate
    res = [exp[k] * data[k] for k in range(len(data))]
    return(res,tps)

In [933]:
def filtre_voix_lin(data,sr,maxi,duration):

    debut = maxi - duration//2
    fin = maxi + duration //2
    imax_int = len(data)*maxi//100
    i_deb = imax_int - duration*sr//2
    i_fin = imax_int + duration*sr//2
    lin = []
    for i in range(len(data)):
        if i<i_deb:
            lin.append(0)
        if i>i_deb and i<imax_int:
            lin.append((i-i_deb)/(imax_int-i_deb))
        if i>imax_int and i<i_fin:
            lin.append((i_fin-i)/(i_fin-imax_int))
        else:
            lin.append(0)
    res = [lin[k] * datavoice[k] for k in range(len(data))]
    return np.array(res),debut,fin

In [934]:
#Les enregistrements d'avions et de voix n'ont pas le même sample rate. Afin de pouvoir les additionner facilement, on peut
#modifier le son avec le sample rate le plus petit, en considérant l'intensité constante entre les points 
#effectivement enregistrés

def same_sample_rate(voice,sr_voice,plane,sr_plane):

    if sr_voice>sr_plane:
        voice_adjusted = voice
        pas = int(sr_voice/sr_plane)
        plane_adjusted = []
        for t in range(len(plane)):
            plane_adjusted += [plane[t] for p in range(pas)]
    else:
        plane_adjusted = plane
        pas = int(sr_plane/sr_voice)
        for t in range(len(voice)):
            voice_adjusted += [voice[t] for p in range(pas)]
    return (np.array(voice_adjusted),np.array(plane_adjusted),max([sr_voice,sr_plane]))

In [935]:
def superpose(plane,voice,debut): #superpose deux sons de meme samplerate

    percent_voice = len(voice)/len(plane) * 100
    if percent_voice > 100-debut:
        debut_effectif = 99 - debut #on decale si ca fait trop

    else:
        debut_effectif = debut
    plane = np.array(plane)
    voice = np.array(voice)

    len_duration = len(voice)

    t_debut = int(debut/100 * len(plane))
    print(t_debut)
    t_fin = t_debut + len_duration
    print(t_fin)
    res = plane
    # print(len(res),len_duration)
    res[t_debut : t_fin] = np.add(plane[t_debut : t_fin],voice)

    return res,debut_effectif

In [936]:
def get_voice_sec_interval(plane,sr,debut,voice):
    signal_duration = len(plane)/sr 
    sec_debut = debut/100*signal_duration

    percent_voice = len(voice)/len(plane) * 100
    fin = debut + percent_voice

    sec_fin = fin/100*signal_duration
    return sec_debut,sec_fin

def get_filter_sec_interval(s_debut_voix,s_fin_voix,debut_filtre,fin_filtre):
    
    voice_duration = s_fin_voix-s_debut_voix
    sec_debut_filtre = s_debut_voix+ debut_filtre/100*voice_duration
    sec_fin_filtre = s_debut_voix+ fin_filtre/100*voice_duration

    return sec_debut_filtre,sec_fin_filtre

In [937]:
def generate_database(nb):
    db_infos = {}
    for i in range(nb):
        #on selectionne les fichiers
        path_voice,file = random_voice_file()
        path_plane = random_plane_file()
        
        #on les ouvre
        datavoice, sr_voice = sf.read(path_voice)
        dataplane, sr_plane = sf.read(path_plane)

        voice_duration = len(datavoice)/sampleratevoice

        #on applique un filtre sur la voix ou pas
        # maxi_filtre = random.randint(0,100) #intensite max du filtre
        # duration_filtre = random.randint(0,25)
        # voice_filtered,debut_filtre,fin_filtre = filtre_voix_lin(datavoice,sr_voice,maxi_filtre,duration_filtre)

        #on les met a la meme frequence d'echantillonage
        plane,voice,sr = same_sample_rate(dataplane,sr_plane,datavoice,sr_voice)

        debut = random.randint(0,100) # en pourcent de l'extrait
        print(debut)
        #on les superpose à partir de debut
        sum_sounds,debut_effectif = superpose(plane,voice,debut) #il y aura 10% de voix
        
        name =  "db\\sample_{}.wav".format(i)

        sec_voice_debut,sec_voice_fin = get_voice_sec_interval(sum_sounds,sr,debut_effectif,voice)
        # sec_filter_debut,sec_filter_fin = get_filter_sec_interval(sec_voice_debut,sec_voice_fin,debut_filtre,fin_filtre)
        db_infos[i] = {
            'debut_voix':sec_voice_debut,
            'fin_voix':sec_voice_fin,
            # 'filtre':{
            #     'debut':sec_filter_debut,
            #     'max':(sec_filter_debut+sec_filter_fin)//2,
            #     'fin':sec_filter_fin
            # }
        }

        sf.write(name, sum_sounds, sr)

    return db_infos

In [938]:
db_infos = generate_database(1)

21
1651507
2122867


In [939]:
print(db_infos)

{0: {'debut_voix': 50.4, 'fin_voix': 64.784765625}}
